In [26]:
import os
from typing import Optional

import scipy

# from mido import MidiFile
import networkx as nx
from networkx_viewer import Viewer
import matplotlib
import matplotlib.pyplot as plt

import math
import numpy as np
import pandas as pd

from torch_geometric.data import HeteroData


print(scipy.__version__)
print(matplotlib.__version__)
print(nx.__version__)

1.7.3
3.6.2
2.8.4


In [27]:
def complete_graph(input_path) -> nx.Graph:
    edgelists = [qf for qf in os.listdir(input_path)
                 if qf.endswith('.edgelist') and not qf.startswith('_')]
    g = None

    print('loading edgelists...')
    for eg in edgelists:
        print('- ' + eg)
        h = nx.read_edgelist(os.path.join(input_path, eg), nodetype=str, create_using=nx.DiGraph(), delimiter=' ')
        for edge in h.edges():
            h[edge[0]][edge[1]]['weight'] = 1

        g = h if g is None else nx.compose(g, h)

    g = g.to_undirected()

    print('Nodes: %d' % nx.number_of_nodes(g))
    print('Edges: %d' % nx.number_of_edges(g))
    return g


In [28]:
G = complete_graph(".\slac\embeddings\Test Edgelists")

loading edgelists...
- full_edgelist.edgelist
- notes.edgelist
- program.edgelist
- tempo.edgelist
- time.signature.edgelist
Nodes: 2703
Edges: 19366


In [29]:
nodes = pd.DataFrame((list(G.nodes)), columns=['name'])
edges = pd.DataFrame(np.array(list(G.edges)), columns=['source', 'target'])

In [30]:
note_groups = [n for n in nodes['name'] if n[0] == 'g' and n[1] in [str(i) for i in range(10)] + ['-'] ]

not_group_nodes = [n for n in nodes['name'] if n not in note_groups]

url = [n for n in not_group_nodes if n[:4] == 'http']
program_nodes = []
note_nodes = []
for u in url:
    if "programs" in u:
        program_nodes.append(u)
    elif "notes" in u:
        note_nodes.append(u)
    else:
        print(u)

name_nodes = [n for n in not_group_nodes if n[0] == '-']
dur_nodes = [n for n in not_group_nodes if n[:3] == 'dur']
vel_nodes = [n for n in not_group_nodes if n[:3] == 'vel']
time_nodes = [n for n in not_group_nodes if n[:4] == 'time']
tempo_nodes = [n for n in not_group_nodes if n not in set(dur_nodes).union(vel_nodes, time_nodes, name_nodes, url)]

tempo_nodes

['11', '6', '9']

In [31]:
node_categories = {"note_group": note_groups,
                    "pitch": note_nodes,
                    "program": program_nodes,
                    "MIDI": name_nodes,
                    "duration": dur_nodes,
                    "velocity": vel_nodes,
                    "time_sig": time_nodes,
                    "tempo": tempo_nodes
                   }

node_categories.keys()

dict_keys(['note_group', 'pitch', 'program', 'MIDI', 'duration', 'velocity', 'time_sig', 'tempo'])

In [32]:
def add_edge_names(edges_pd: pd.DataFrame, node_cat: dict) -> pd.DataFrame:
    edge_type = []

    edge_name_source = None
    edge_name_target = None

    augmented_edges_pd = edges_pd.copy()

    for i in range(len(edges.index)):
        for name in node_cat.keys():
            if edges.iloc[i]['source'] in node_cat[name]:
                edge_name_source = name + "__"
                break
        for name in node_cat.keys():
            if edges.iloc[i]['target'] in node_cat[name]:
                edge_name_target = name
                break
                
        edge_name = edge_name_source + edge_name_target
        edge_type.append(edge_name)
        
    augmented_edges_pd['edge_type'] = edge_type
    return augmented_edges_pd


In [33]:
edges_2 = add_edge_names(edges, node_categories)


In [34]:
set(edges_2['edge_type'])

{'MIDI__note_group',
 'MIDI__program',
 'MIDI__tempo',
 'MIDI__time_sig',
 'duration__note_group',
 'note_group__MIDI',
 'note_group__duration',
 'note_group__pitch',
 'note_group__velocity',
 'pitch__note_group',
 'program__MIDI',
 'time_sig__MIDI',
 'velocity__note_group'}

In [35]:
edges_2.loc[edges_2['edge_type'] == 'MIDI__program', ['source', 'target']]

,source,target
933,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/30
934,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/0
935,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/33
9697,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/17
9698,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/4
9699,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/26
9700,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/61
9701,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/56
9702,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/66
9703,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/35


In [36]:
edges_2.loc[edges_2['edge_type'] == 'program__MIDI', ['source', 'target']]

,source,target
7102,http://purl.org/midi-ld/programs/30,-B_B_King_-_Rock_Me_Baby
7103,http://purl.org/midi-ld/programs/0,-B_B_King_-_How_Blue_Can_You_Get
7104,http://purl.org/midi-ld/programs/0,-B_B_King_-_Rock_Me_Baby
7105,http://purl.org/midi-ld/programs/33,-B_B_King_-_Rock_Me_Baby


In [37]:
main_edge_types = ["MIDI__has__tempo",
                   "MIDI__in__time_sig",
                   "MIDI__has__program",
                   "MIDI__has__note_group",
                   "note_group__has__velocity",
                   "note_group__has__duration",
                   "note_group__contains__pitch"]


# 'duration__note_group',
# 'note__note_group',
# 'velocity__note_group'

# 'note_group__MIDI',
# 'program__MIDI',
# 'time_sig__MIDI',


In [38]:
def reverse_edge(df: pd.DataFrame, row: int, inplace: bool = False) -> Optional[pd.DataFrame]:
    if inplace:
        df.iloc[row]['source'], df.iloc[row]['target'] = df.iloc[row]['target'], df.iloc[row]['source']
        return None
    elif not inplace:
        tmp = df.copy()
        tmp.iloc[row]['source'], tmp.iloc[row]['target'] = tmp.iloc[row]['target'], tmp.iloc[row]['source']
        return tmp


def format_edge_name(source: str, target: str) -> str:
    edge_name = ""

    if source == "MIDI":
        if target == "tempo":
            edge_name = source + "__has__" + target
        elif target == "time_sig":
            edge_name = source + "__in__" + target
        elif target == "program":
            edge_name = source + "__has__" + target
        elif target == "note_group":
            edge_name = source + "__has__" + target
    elif source == "note_group":
        if target == "velocity":
            edge_name = source + "__has__" + target
        elif target == "duration":
            edge_name = source + "__has__" + target
        elif target == "pitch":
            edge_name = source + "__contains__" + target
    else:
        edge_name = source + "__?__" + target
        print("Not known edge detected: " + edge_name)
        return edge_name

    return edge_name
    

def add_edge_names2(edges_df: pd.DataFrame, node_cat: dict) -> pd.DataFrame:
    edge_type = []

    edge_name_source = ""
    edge_name_target = ""

    augmented_edges_df = edges_df.copy()

    for i in range(len(edges_df.index)):
        for name in node_cat.keys():
            if edges_df.iloc[i]['source'] in node_cat[name]:
                edge_name_source = name
                break
        for name in node_cat.keys():
            if edges_df.iloc[i]['target'] in node_cat[name]:
                edge_name_target = name
                break

        if (edge_name_source not in ("MIDI", "note_group")) or (edge_name_source == "note_group" and edge_name_target == "MIDI"):
            reverse_edge(augmented_edges_df, row=i, inplace=True)
            edge_name_source, edge_name_target = edge_name_target, edge_name_source

        edge_name = format_edge_name(edge_name_source, edge_name_target)
        edge_type.append(edge_name)

    augmented_edges_df['edge_type'] = edge_type
    return augmented_edges_df


In [39]:
edges_3 = add_edge_names2(edges, node_categories)

In [40]:
set(edges_3['edge_type'])

{'MIDI__has__note_group',
 'MIDI__has__program',
 'MIDI__has__tempo',
 'MIDI__in__time_sig',
 'note_group__contains__pitch',
 'note_group__has__duration',
 'note_group__has__velocity'}

In [41]:
edges_3.loc[edges_3['edge_type'] == "MIDI__has__program", ['source', 'target', ]]

,source,target
933,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/30
934,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/0
935,-Albert_King_-_Born_Under_A_Bad_Sign,http://purl.org/midi-ld/programs/33
7102,-B_B_King_-_Rock_Me_Baby,http://purl.org/midi-ld/programs/30
7103,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/0
7104,-B_B_King_-_Rock_Me_Baby,http://purl.org/midi-ld/programs/0
7105,-B_B_King_-_Rock_Me_Baby,http://purl.org/midi-ld/programs/33
9697,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/17
9698,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/4
9699,-B_B_King_-_How_Blue_Can_You_Get,http://purl.org/midi-ld/programs/26


In [55]:
def add_node_cat(nodes_df: pd.DataFrame, node_cat: dict) -> pd.DataFrame:
    node_type = []
    augumented_nodes_df = nodes_df.copy()
    for i in range(len(nodes_df.index)):
        for key in node_cat.keys():
            if nodes.iloc[i]['name'] in node_cat[key]:
                node_type.append(key)
    augumented_nodes_df['node_type'] = node_type
    
    return augumented_nodes_df


In [56]:
nodes_2 = add_node_cat(nodes, node_categories)

In [57]:
set(nodes_2["node_type"])

{'MIDI',
 'duration',
 'note_group',
 'pitch',
 'program',
 'tempo',
 'time_sig',
 'velocity'}

In [58]:
def add_types(nodes_df: pd.DataFrame, edges_df: pd.DataFrame, node_cat: dict) -> (pd.DataFrame, pd.DataFrame):
    return add_node_cat(nodes_df, node_categories), add_edge_names2(edges_df, node_categories)

In [59]:
nodes_f, edges_f = add_types(nodes, edges, node_categories)